# Einleitung

In diesem Notebook vergleichen wir verschiedene Ansätze, um sicherzustellen, dass unser Bot die am besten passenden Antworten generiert.

Die 5 Fragen, die verglichen sollen folgende Eigenschaften besitzen:
- Die Fragen sollen Fakultät übergreifend sein
- Die Fragen sollen TH-spezifisch sein
- Die Fragen sollen TH-Intranet-spezifisch sein
- Die Fragen sollen Realitätsnah sein

Die 5 Fragen, die verglichen werden sind:

1. Wie ist die Email Adresse von Professor Gallwitz?
2. Was soll ich beachten, wenn ich eine Prüfung anmelden will?
3. Welche voraussetzungen, muss ich für den Mater Studiengang erfüllen?
4. Welche Professoren gibt es an der Fakultät Soziale Arbeit?
5. Wann und was muss ich im IT-Projekt machen?

## Beispiel Tabelle

Nun werden fogende Parameter verglichen.

| LLM Modell    | Kontext Variation | Word Embedding Model | Frage 1 | Antwort 1 | Frage 2 | Antwort 2 | Frage 3 | Antwort 3 | Frage 4 | Antwort 4 | Frage 5 | Antwort 5 | Durchschnitt |
|---------------|-------------------|----------------------|---------|-----------|---------|-----------|---------|-----------|---------|-----------|---------|-----------|--------------|
| Vicuna groß   | 1 Dokument        | a(MiniLM)            | 2       |           | 4       |           | 5       |           | 8       |           | 7       |           | 5.2          |
| Vicuna groß   | 1 Dokument        | b(xy)                 | 10      |           | 10      |           | 10      |           | 10      |           | 10      |           | 10           |
| Vicuna groß   | 1 Dokument        | c(TF-IDF)             |         |           |         |           |         |           |         |           |         |           | 0            |
| Vicuna groß   | 1 Dokument        | d(MiniLM*TF-IDF)      |         |           |         |           |         |           |         |           |         |           | 0            |
| Vicuna groß   | 1 Dokument        | e(xy*TF-IDF)          |         |           |         |           |         |           |         |           |         |           | 0            |
| Vicuna groß   | 5 Dokumente       | a                     |         |           |         |           |         |           |         |           |         |           | 0            |
| Vicuna groß   | 5 Dokumente       | b                     |         |           |         |           |         |           |         |           |         |           | 0            |
| Vicuna groß   | 5 Dokumente       | c                     |         |           |         |           |         |           |         |           |         |           | 0            |
| Vicuna groß   | 5 Dokumente       | d                     |         |           |         |           |         |           |         |           |         |           | 0            |
| Vicuna groß   | 5 Dokumente       | e                     |         |           |         |           |         |           |         |           |         |           | 0            |
| Vicuna groß   | 10 Dokumente      | a                     |         |           |         |           |         |           |         |           |         |           | 0            |
| Vicuna groß   | 10 Dokumente      | b                     |         |           |         |           |         |           |         |           |         |           | 0            |
| Vicuna groß   | 10 Dokumente      | c                     |         |           |         |           |         |           |         |           |         |           | 0            |
| Vicuna groß   | 10 Dokumente      | d                     |         |           |         |           |         |           |         |           |         |           | 0            |
| Vicuna groß   | 10 Dokumente      | e                     |         |           |         |           |         |           |         |           |         |           | 0            |


Nun soll ein script ersteltt werden, der die Antwort Spalten generiert.
Nacher werden die Antworten manuell bewertet von allen Projekt Teilnehmer.

## Imports

In [1]:
import sys

sys.path.append("..")
sys.path.append("../..")
from scrap.query_crafter import construct_prompt
from scrap.embedding_algorithms.tdIdfDistance import get_most_similar_articles_tf_idf
from scrap.embedding_algorithms.question_embedding_MiniLM import (
    get_most_similar_articles_MiniLM,
)
from LLM.ServerCommunicator import server_get_answer
from dotenv import load_dotenv

load_dotenv()
import os
import torch
from huggingface_hub import login
from transformers import pipeline  # loading of hf LLMs
import gc
import pandas as pd

/Users/br/Projects/IT-Ptojekt-Chatbot/daibl-slim/daibl/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Verschiedene Embedding Modelle für Kontextgenerierung

In [ ]:
def get_most_similar_articles_for_specified_embeding_model(embedding_model,question,document_amount):
    
    embeddings_model_dict = {
        'MiniLM': get_most_similar_articles_MiniLM,
        'TF-IDF': get_most_similar_articles_tf_idf
    }
     
    get_most_similar_articles = embeddings_model_dict.get(embedding_model)
    result = get_most_similar_articles(question,document_amount)
    
    return result

### LLM Model laden 

In [ ]:

def model_load(model_id):
  HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")
  login(token=HUGGINGFACEHUB_API_TOKEN)
  device = "cuda:0" if torch.cuda.is_available() else "cpu"

  model = pipeline(
    task="text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map='auto',
    temperature=0.3,
    top_p=0.15,
    top_k=15,
    repetition_penalty=1.1,
    num_return_sequences=1,
    max_new_tokens=128,
    #max_length=256,
  )
  return model


In [ ]:
def model_unload(model):
    del model
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:

def return_prompt_answer(model, query_prompt):
    """ process and return answer of LLM """
    
    answer = model(query_prompt, do_sample=True)
    return answer[0]['generated_text']

Make sure to forward the port to the cluster for the vicuna_70b model.
To do so, copy the template into the terminal, change the username:  
> ssh -N -L localhost:8080:localhost:8087 \<mustermannm12345\>@141.75.89.6 

and enter your password afterwards.
The Terminal has to be kept open for the tunneling to work.


In [ ]:
async def get_answer_from_model(model_id, query_prompt,model):
    if model_id == 'vicuna_70b':
        answer = server_get_answer(query_prompt)
    elif model_id == 'lmsys/vicuna-13b-v1.5': 
        answer = return_prompt_answer(model,query_prompt)
    elif model_id  == 'meta-llama/Llama-2-13b-chat-hf':
        answer = return_prompt_answer(model,query_prompt)

    return answer


In [ ]:
async def generate_response(model_id,document_amount,embedding_model,question,model):
    anwser=""
    documents = get_most_similar_articles_for_specified_embeding_model(embedding_model,question,document_amount)
    query_prompt = construct_prompt(documents, question) 
    anwser = await get_answer_from_model(model_id,query_prompt,model)
    
    return anwser

### Test

In [ ]:
await generate_response("vicuna_70b",1,"MiniLM","Wie ist die Email Adresse von Professor Gallwitz?")

## Alle Variationen

In [ ]:
import pandas as pd

columns = ["Model", "Document Amount", "Embeddings Model", "Question", "Response"]
df = pd.DataFrame(columns=columns)

model_ids = ["vicuna_70b","vicuna_13b","meta-llama/Llama-2-13b-chat-hf"]
document_amounts = [1, 5, 10]
embeddings_models=["MiniLM","TF-IDF"]
questions=["Wie ist die Email Adresse von Professor Gallwitz?",
           "Was soll ich beachten, wenn ich eine Prüfung anmelden will?",
           "Welche voraussetzungen, muss ich für den Master Studiengang erfüllen?",
           "Welche Professoren gibt es an der Fakultät Soziale Arbeit?",
           "Wann und was muss ich im IT-Projekt machen?"]

data = []
model= None
for model_id in model_ids:
    if(model_id != "vicuna_70b"):
       model= model_load(model_id)
    for document_amount in document_amounts:
        for embeddings_model in embeddings_models:
            for question in questions:
                response = await generate_response(model_id, document_amount, embeddings_model, question,model)
                data.append([model_id, document_amount, embeddings_model, question, response])
    

df = pd.concat([df, pd.DataFrame(data, columns=columns)], ignore_index=True)

df.to_excel('Evaluation.xlsx', index=False)


In [ ]:
model_id="meta-llama/Llama-2-13b-chat-hf"
model_id=model_load(model_id)
####################### Pipeline for model loading and unloading ###############################
import os
import gc
from multiprocessing import Process

import torch
from huggingface_hub import login
from transformers import pipeline 

def run_model(model_id, question):
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    print(f"Torch allocated memory: {torch.cuda.memory_allocated()}; {device}")
    model = pipeline(
        task="text-generation", 
        model=model_id,
        torch_dtype=torch.bfloat16, 
        device_map='auto',
        temperature=0.3,
        top_p=0.15,
        top_k=15,
        repetition_penalty=1.1,
        num_return_sequences=1,
        #max_new_tokens=128,
        max_length=64,
    )
    answer = model(question, do_sample=True)
    print(answer[0]['generated_text'])
    
    # free resources and memory after process terminates
    del model
    gc.collect()
    torch.cuda.empty_cache()
    print(f"Torch allocated memory: {torch.cuda.memory_allocated()}; {device}")

if __name__ == "__main__":
    login(token="token")

    question = "Hallo wer bist du?"

    model_id = ["meta-llama/Llama-2-13b-chat-hf", "meta-llama/Llama-2-7b-chat-hf", "meta-llama/Llama-2-7b-chat-hf"]
    for i in range(2):
        for name in model_id:
            p = Process(target=run_model, args=(name, question))
            p.start()
            p.join()

## Daten automatisch evaluieren lassen

In [3]:
import sys

sys.path.append("..")
import pandas as pd
from scrap.db_init import db_get_df, db_save_df

In [ ]:
file_path = "~/Desktop/output.xlsx"
df = pd.read_excel(file_path)
print(df.dtypes)

In [ ]:
db_save_df(df, "evaluation")

In [ ]:
def get_sample_answer(question):
    if question == "Wie ist die Email Adresse von Professor Gallwitz?":
        return "Die Email Adresse von Prof. Dr. Gallwitz ist: florian.gallwitz@th-nuernberg.de"
    elif question == "Was soll ich beachten, wenn ich eine Prüfung anmelden will?":
        return "Um eine Prüfung anzumelden, müssen Sie folgende Kriterien beachten: Anmeldefristen,Zulassungsvoraussetzungen, Anmeldeverfahren, Prüfungstermin, Prüfungsvorbereitung."
    elif (
        question
        == "Welche voraussetzungen, muss ich für den Master Studiengang erfüllen?"
    ):
        return "Für den MIN Master Studiengang an der Technische Hochschule Nürnberg , müssen Sie folgende Voraussetzungen erfüllen: Bachelorabschluss in IN/MIN/WIN oder verwandete Fächer, Notendurchschnitt von 2,5, Sprachkenntnisse von C1, Bewerbungsunterlagen bereit stellen."
    elif question == "Welche Professoren gibt es an der Fakultät Soziale Arbeit?":
        return "An der Fakultät für Soziale Arbeit gibt es folgende Proffessoren: Johannes Bach, Steffen Brockmann, Michael Domes, Simone Emmert, Carolin Freier, Sabine Fromm …"
    elif question == "Wann und was muss ich im IT-Projekt machen?":
        return "Der Praxisbeauftragter der Fakultät Informatik ist Prof. Dr. Wolfgang Bremer."

In [ ]:
for row in df.iterrows():
    row = row[1]
    question = row["Question"]
    sample_answer = get_sample_answer(question)
    model_answer = row["Response"]
    print(question, sample_answer, model_answer)

In [22]:
from gpt_evaluate import gpt_evaluate_outputs

df = db_get_df("evaluation")
df = gpt_evaluate_outputs(df)

60it [02:18,  2.30s/it]


In [ ]:
df["Score"]

In [29]:
for index, row in df.iterrows():
    if isinstance(row['Score'], dict):
        s1 = row['Score'].get('Inhaltliche Korrektheit', 0)
        s2 = row['Score'].get('Sprachliche Gestaltung', 0)
        s3 = row['Score'].get('Fokus', 0)
        df.at[index, 'Score'] = s1 + s2 + s3

In [32]:
df = df.drop('Begründung', axis=1)
df.head()

,Model,Document Amount,Embeddings Model,Question,Response,Score
0,lmsys/vicuna-13b-v1.5,1,MiniLM,Wie ist die Email Adresse von Professor Gallwitz?,\n \n Die Antwort lautet: [galwi...,7
1,lmsys/vicuna-13b-v1.5,1,MiniLM,"Was soll ich beachten, wenn ich eine Prüfung a...",\n\n Antwort: B) Unterlagen für die Anm...,2
2,lmsys/vicuna-13b-v1.5,1,MiniLM,"Welche voraussetzungen, muss ich für den Maste...","\n \n Um produktiv zu bleiben, h...",5
3,lmsys/vicuna-13b-v1.5,1,MiniLM,Welche Professoren gibt es an der Fakultät Soz...,"\n \n Es tut mir leid, aber ich ...",1
4,lmsys/vicuna-13b-v1.5,1,MiniLM,Wann und was muss ich im IT-Projekt machen?,\n \n IT-Projekt:\n Proje...,7


In [33]:
db_save_df(df, "evaluation_gpt3.5")


In [7]:
df = db_get_df("'evaluation_gpt3.5'")

In [8]:
df.to_excel('Evaluation_gpt3.5.xlsx', index=False)